In [10]:
# 📦 Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from surprise import Dataset, Reader
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import os
import warnings
warnings.filterwarnings("ignore")

# 🌍 Introduction
print("\U0001F30D Système de recommandation pour l'artisanat marocain")

# 📊 Chargement des données
print("\n\U0001F4CA Chargement des données...")
try:
    with open('../data/processed/clean_dataset.pkl', 'rb') as f:
        df = pickle.load(f)
    print("Données chargées depuis le fichier pickle.")
except:
    df = pd.read_csv('../data/processed/clean_dataset.csv')
    print("Données chargées depuis le fichier CSV.")

# 🔍 Filtrer les produits d'artisanat marocain
print("\n\U0001F50D Filtrage des produits d'artisanat marocain...")
keywords = [
    'morocco', 'moroccan', 'maroc', 'marocain', 'marocaine', 'fez', 'casablanca', 'marrakech',
    'berber', 'berbère', 'atlas', 'maghreb', 'tanger', 'rabat', 'agadir', 'essaouira',
    'tapis', 'carpet', 'pottery', 'ceramic', 'leather', 'cuir', 'babouche', 'tajine', 'tagine',
    'lantern', 'lanterne', 'pouf', 'zellige', 'caftan', 'kaftan', 'djellaba', 'jellaba'
]

def is_moroccan_craft(text):
    if pd.isna(text):
        return False
    text = text.lower()
    return any(keyword in text for keyword in keywords)

df['is_moroccan'] = df['Full_Text'].apply(is_moroccan_craft)
moroccan_df = df[df['is_moroccan']]

# ✅ Réinitialiser l'index pour correspondre à la matrice TF-IDF
moroccan_df = moroccan_df.reset_index(drop=True)
products_df = df.drop_duplicates('Product Id').reset_index(drop=True)

print(f"Nombre de produits d'artisanat marocain identifiés: {len(moroccan_df)}")

# 🔄 Chargement des modèles
print("\n\U0001F501 Chargement des modèles...")
try:
    with open('../models/collaborative/svd_model.pkl', 'rb') as f:
        svd_model = pickle.load(f)
    svd_loaded = True
    print("Modèle SVD chargé avec succès.")
except:
    svd_loaded = False
    print("Erreur: SVD non chargé.")

try:
    with open('../models/content_based/tfidf_vectorizer.pkl', 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    with open('../models/content_based/tfidf_knn_model.pkl', 'rb') as f:
        tfidf_knn_model = pickle.load(f)
    tfidf_loaded = True
    print("TF-IDF + KNN chargé.")
except:
    tfidf_loaded = False

try:
    with open('../models/content_based/bert_embeddings.pkl', 'rb') as f:
        bert_embeddings = pickle.load(f)
    with open('../models/content_based/bert_knn_model.pkl', 'rb') as f:
        bert_knn_model = pickle.load(f)
    bert_loaded = True
    print("BERT + KNN chargé.")
except:
    bert_loaded = False

# 🔮 Fonction de recommandation
def get_recommendations(product_id, user_id=None, n=5):
    recommendations = {}
    product_info = moroccan_df[moroccan_df['Product Id'] == product_id].iloc[0]
    product_idx_moroccan = moroccan_df[moroccan_df['Product Id'] == product_id].index[0]
    product_idx_products = products_df[products_df['Product Id'] == product_id].index[0]

    if svd_loaded and user_id is not None:
        svd_recs = []
        for pid in moroccan_df['Product Id'].unique():
            if pid != product_id:
                try:
                    pred = svd_model.predict(user_id, pid)
                    svd_recs.append((pid, pred.est))
                except:
                    pass
        svd_recs.sort(key=lambda x: x[1], reverse=True)
        recommendations['svd'] = [
            (pid, score, moroccan_df[moroccan_df['Product Id'] == pid]['Product Name'].iloc[0])
            for pid, score in svd_recs[:n]
        ]

    if tfidf_loaded:
        tfidf_matrix = tfidf_vectorizer.transform(moroccan_df['Full_Text'])
        product_vector = tfidf_matrix[product_idx_moroccan].reshape(1, -1)
        distances, indices = tfidf_knn_model.kneighbors(product_vector)
        tfidf_recs = []
        for i, idx in enumerate(indices[0][1:n+1]):
            similar_product = moroccan_df.iloc[idx]
            tfidf_recs.append((
                similar_product['Product Id'],
                1 - distances[0][i+1],
                similar_product['Product Name']
            ))
        recommendations['tfidf'] = tfidf_recs

    if bert_loaded:
        product_vector = bert_embeddings[product_idx_products].reshape(1, -1)
        distances, indices = bert_knn_model.kneighbors(product_vector)
        bert_recs = []
        for i, idx in enumerate(indices[0][1:n+1]):
            similar_product = products_df.iloc[idx]
            if similar_product['Product Id'] in moroccan_df['Product Id'].values:
                bert_recs.append((
                    similar_product['Product Id'],
                    1 - distances[0][i+1],
                    similar_product['Product Name']
                ))
        recommendations['bert'] = bert_recs

    return recommendations, product_info

# 🎯 Démonstration
sample_product_id = moroccan_df['Product Id'].sample(1).iloc[0]
sample_user_id = df['Uniq Id'].sample(1).iloc[0]
recommendations, product_info = get_recommendations(sample_product_id, sample_user_id)

print(f"\nProduit sélectionné: {product_info['Product Name']}")
print(f"Catégorie: {product_info['Product Category']}")
print(f"Description: {product_info['Product Description'][:200]}...")

if 'svd' in recommendations:
    print("\n\U0001F4CA Recommandations par SVD:")
    for i, (pid, score, name) in enumerate(recommendations['svd']):
        print(f"{i+1}. {name} (Score: {score:.4f})")
if 'tfidf' in recommendations:
    print("\n\U0001F4CA Recommandations par TF-IDF:")
    for i, (pid, score, name) in enumerate(recommendations['tfidf']):
        print(f"{i+1}. {name} (Similarité: {score:.4f})")
if 'bert' in recommendations:
    print("\n\U0001F4CA Recommandations par BERT:")
    for i, (pid, score, name) in enumerate(recommendations['bert']):
        print(f"{i+1}. {name} (Similarité: {score:.4f})")


🌍 Système de recommandation pour l'artisanat marocain

📊 Chargement des données...
Données chargées depuis le fichier pickle.

🔍 Filtrage des produits d'artisanat marocain...
Nombre de produits d'artisanat marocain identifiés: 107

🔁 Chargement des modèles...
Modèle SVD chargé avec succès.
TF-IDF + KNN chargé.
BERT + KNN chargé.


IndexError: single positional indexer is out-of-bounds

In [2]:
# 🌍 Introduction  
print("🌍 Système de recommandation pour l'artisanat marocain")  
print("Ce notebook démontre l'application de notre système de recommandation sur des produits d'artisanat marocain.")  
print("Nous utiliserons les modèles entraînés précédemment pour générer des recommandations personnalisées.")  
  
# 📊 Chargement des données  
print("\n📊 Chargement des données...")  
try:  
    # Essayer de charger depuis le fichier pickle  
    with open('../data/processed/clean_dataset.pkl', 'rb') as f:  
        df = pickle.load(f)  
    print("Données chargées depuis le fichier pickle.")  
except:  
    # Sinon, charger depuis le CSV  
    df = pd.read_csv('../data/processed/clean_dataset.csv')  
    print("Données chargées depuis le fichier CSV.")  
  

🌍 Système de recommandation pour l'artisanat marocain
Ce notebook démontre l'application de notre système de recommandation sur des produits d'artisanat marocain.
Nous utiliserons les modèles entraînés précédemment pour générer des recommandations personnalisées.

📊 Chargement des données...
Données chargées depuis le fichier pickle.


In [3]:
# 🔍 Filtrer les produits d'artisanat marocain  
print("\n🔍 Filtrage des produits d'artisanat marocain...")  
# Mots-clés pour identifier les produits d'artisanat marocain  
keywords = ['morocco', 'moroccan', 'maroc', 'marocain', 'marocaine', 'fez', 'casablanca', 'marrakech',   
            'berber', 'berbère', 'atlas', 'maghreb', 'tanger', 'rabat', 'agadir', 'essaouira',  
            'tapis', 'carpet', 'pottery', 'ceramic', 'leather', 'cuir', 'babouche', 'tajine', 'tagine',  
            'lantern', 'lanterne', 'pouf', 'zellige', 'caftan', 'kaftan', 'djellaba', 'jellaba']  
  
# Fonction pour vérifier si un produit est lié à l'artisanat marocain  
def is_moroccan_craft(text):  
    if pd.isna(text):  
        return False  
    text = text.lower()  
    return any(keyword in text for keyword in keywords)  
  
# Filtrer les produits  
df['is_moroccan'] = df['Full_Text'].apply(is_moroccan_craft)  
moroccan_df = df[df['is_moroccan']]  
  
print(f"Nombre de produits d'artisanat marocain identifiés: {len(moroccan_df)}")  
  
# Si aucun produit n'est trouvé, utiliser un sous-ensemble aléatoire pour la démonstration  
if len(moroccan_df) < 10:  
    print("Peu de produits d'artisanat marocain trouvés. Utilisation d'un sous-ensemble aléatoire pour la démonstration.")  
    moroccan_df = df.sample(100)  


🔍 Filtrage des produits d'artisanat marocain...
Nombre de produits d'artisanat marocain identifiés: 107


In [4]:
# 📊 Aperçu des produits d'artisanat marocain  
print("\n📊 Aperçu des produits d'artisanat marocain:")  
display(moroccan_df[['Product Name', 'Product Category', 'Product Brand']].head(10))  
  
# 🔄 Chargement des modèles  
print("\n🔄 Chargement des modèles...")  
  
# Chargement du modèle SVD (filtrage collaboratif)  
try:  
    with open('../models/collaborative/svd_model.pkl', 'rb') as f:  
        svd_model = pickle.load(f)  
    print("Modèle SVD chargé avec succès.")  
    svd_loaded = True  
except:  
    print("Erreur: Impossible de charger le modèle SVD.")  
    svd_loaded = False  
  
# Chargement du modèle KNN (filtrage collaboratif)  
try:  
    with open('../models/collaborative/knn_model.pkl', 'rb') as f:  
        knn_collab_model = pickle.load(f)  
    print("Modèle KNN (filtrage collaboratif) chargé avec succès.")  
    knn_collab_loaded = True  
except:  
    print("Erreur: Impossible de charger le modèle KNN (filtrage collaboratif).")  
    knn_collab_loaded = False  
  
# Chargement du vectoriseur TF-IDF et du modèle KNN associé  
try:  
    with open('../models/content_based/tfidf_vectorizer.pkl', 'rb') as f:  
        tfidf_vectorizer = pickle.load(f)  
    with open('../models/content_based/tfidf_knn_model.pkl', 'rb') as f:  
        tfidf_knn_model = pickle.load(f)  
    print("Modèle TF-IDF + KNN chargé avec succès.")  
    tfidf_loaded = True  
except:  
    print("Erreur: Impossible de charger le modèle TF-IDF + KNN.")  
    tfidf_loaded = False  
  
# Chargement des embeddings BERT et du modèle KNN associé  
try:  
    with open('../models/content_based/bert_embeddings.pkl', 'rb') as f:  
        bert_embeddings = pickle.load(f)  
    with open('../models/content_based/bert_knn_model.pkl', 'rb') as f:  
        bert_knn_model = pickle.load(f)  
    print("Modèle BERT + KNN chargé avec succès.")  
    bert_loaded = True  
except:  
    print("Erreur: Impossible de charger le modèle BERT + KNN.")  
    bert_loaded = False  


📊 Aperçu des produits d'artisanat marocain:


,Product Name,Product Category,Product Brand
11,"ORLY Nail Lacquer for Women, #20634 Red Carpet...",Beauty > Nail Care > Nail Polish > Nail Lacquer,ORLY
110,"NOW Foods 100% Pure Eucalyptus Oil, 2 Fl Oz",Health > Aromatherapy > Essential Oils > Essen...,NOW
194,Zep Air and Fabric Odor Eliminator 32 Ounce ZU...,Household Essentials > Air Fresheners > Spray ...,Zep
195,Royce Leather Executive Travel Grooming Genuin...,Premium Beauty > Premium Men's Grooming > Prem...,ROYCE New York
201,"CLR Stain Magnet Stain Remover, Eco-Friendly &...",Household Essentials > Bathroom > Bathroom Cle...,CLR
262,EcoTools Cleansing and Exfoliating Bath EcoPou...,"Personal Care > Bath & Body > Loofahs, Bath Sp...",EcoTools
288,"Scotch-Brite Stainless Steel Scrubbing Pads, 3...",Household Essentials > Cleaning Supplies > Cle...,Scotch-Brite
361,Adjust-A-Lift Heel Lift-Medium (Womens 8.5+ Me...,Health > Foot Care > Insoles,Adjust-A-Heel
363,"Disinfectant Restroom Cleaner, Clean Fresh Sce...",Household Essentials > Cleaning Supplies > Dis...,Scrubbing Bubbles
381,2Pair Waterproof Rubber Yellow Reusable Cleani...,Household Essentials > Cleaning Supplies > Cle...,Fymall



🔄 Chargement des modèles...
Modèle SVD chargé avec succès.
Modèle KNN (filtrage collaboratif) chargé avec succès.
Modèle TF-IDF + KNN chargé avec succès.
Modèle BERT + KNN chargé avec succès.


In [5]:
# 🔮 Fonction de recommandation  
def get_recommendations(product_id, user_id=None, n=5):  
    """  
    Génère des recommandations pour un produit donné en utilisant différentes approches.  
      
    Args:  
        product_id: ID du produit pour lequel générer des recommandations  
        user_id: ID de l'utilisateur (optionnel, pour le filtrage collaboratif)  
        n: Nombre de recommandations à générer  
      
    Returns:  
        recommendations: Dictionnaire contenant les recommandations de chaque approche  
    """  
    recommendations = {}  
      
    # Récupération des informations sur le produit  
    product_info = moroccan_df[moroccan_df['Product Id'] == product_id].iloc[0]  
    product_idx = products_df[products_df['Product Id'] == product_id].index[0]  
      
    # Recommandations par filtrage collaboratif (SVD)  
    if svd_loaded and user_id is not None:  
        svd_recs = []  
        # Prédire les notes pour tous les produits pour cet utilisateur  
        for pid in moroccan_df['Product Id'].unique():  
            if pid != product_id:  # Exclure le produit lui-même  
                try:  
                    pred = svd_model.predict(user_id, pid)  
                    svd_recs.append((pid, pred.est))  
                except:  
                    pass  
        # Trier par note prédite décroissante  
        svd_recs.sort(key=lambda x: x[1], reverse=True)  
        recommendations['svd'] = [  
            (pid, score, moroccan_df[moroccan_df['Product Id'] == pid]['Product Name'].iloc[0])  
            for pid, score in svd_recs[:n]  
        ]  

In [6]:
# Recommandations par filtrage basé sur le contenu (TF-IDF)  
    if tfidf_loaded:  
        # Création de la matrice TF-IDF  
        tfidf_matrix = tfidf_vectorizer.transform(moroccan_df['Full_Text'])  
        # Trouver l'index du produit dans la matrice TF-IDF  
        product_idx_in_moroccan = moroccan_df[moroccan_df['Product Id'] == product_id].index[0]  
        # Obtenir le vecteur TF-IDF du produit  
        product_vector = tfidf_matrix[product_idx_in_moroccan].reshape(1, -1)  
        # Trouver les produits similaires  
        distances, indices = tfidf_knn_model.kneighbors(product_vector)  
        # Convertir les indices en IDs de produits  
        tfidf_recs = []  
        for i, idx in enumerate(indices[0][1:n+1]):  # Ignorer le premier qui est le produit lui-même  
            similar_product = moroccan_df.iloc[idx]  
            tfidf_recs.append((  
                similar_product['Product Id'],  
                1 - distances[0][i+1],  
                similar_product['Product Name']  
            ))  
        recommendations['tfidf'] = tfidf_recs  
      
    # Recommandations par filtrage basé sur le contenu (BERT)  
    if bert_loaded:  
        # Trouver l'index du produit dans les embeddings BERT  
        product_idx_in_products = products_df[products_df['Product Id'] == product_id].index[0]  
        # Obtenir l'embedding BERT du produit  
        product_vector = bert_embeddings[product_idx_in_products].reshape(1, -1)  
        # Trouver les produits similaires  
        distances, indices = bert_knn_model.kneighbors(product_vector)  
        # Convertir les indices en IDs de produits  
        bert_recs = []  
        for i, idx in enumerate(indices[0][1:n+1]):  # Ignorer le premier qui est le produit lui-même  
            similar_product = products_df.iloc[idx]  
            # Vérifier si le produit est dans la liste des produits marocains  
            if similar_product['Product Id'] in moroccan_df['Product Id'].values:  
                bert_recs.append((  
                    similar_product['Product Id'],  
                    1 - distances[0][i+1],  
                    similar_product['Product Name']  
                ))  
        recommendations['bert'] = bert_recs  
      
    return recommendations, product_info  

IndentationError: unexpected indent (1451093416.py, line 2)

In [ ]:
# 🎯 Démonstration avec un produit d'artisanat marocain aléatoire  
print("\n🎯 Démonstration avec un produit d'artisanat marocain aléatoire:")  
sample_product_id = moroccan_df['Product Id'].sample(1).iloc[0]  
sample_user_id = df['Uniq Id'].sample(1).iloc[0]  
  
recommendations, product_info = get_recommendations(sample_product_id, sample_user_id)  
  
print(f"\nProduit sélectionné: {product_info['Product Name']}")  
print(f"Catégorie: {product_info['Product Category']}")  
print(f"Description: {product_info['Product Description'][:200]}...")  
  
# Affichage des recommandations  
if 'svd' in recommendations:  
    print("\n📊 Recommandations par filtrage collaboratif (SVD):")  
    for i, (pid, score, name) in enumerate(recommendations['svd']):  
        print(f"{i+1}. {name} (Score: {score:.4f})")  
  
if 'tfidf' in recommendations:  
    print("\n📊 Recommandations par filtrage basé sur le contenu (TF-IDF):")  
    for i, (pid, score, name) in enumerate(recommendations['tfidf']):  
        print(f"{i+1}. {name} (Similarité: {score:.4f})")  
  
if 'bert' in recommendations:  
    print("\n📊 Recommandations par filtrage basé sur le contenu (BERT):")  
    for i, (pid, score, name) in enumerate(recommendations['bert']):  
        print(f"{i+1}. {name} (Similarité: {score:.4f})")  
  
# 🔄 Interface interactive pour tester différents produits  
from IPython.display import clear_output  
  
def interactive_demo():  
    """  
    Interface interactive pour tester le système de recommandation sur différents produits.  
    """  
    while True:  
        clear_output(wait=True)  
        print("🌍 Système de recommandation pour l'artisanat marocain - Démo interactive")  
        print("\n1. Sélectionner un produit aléatoire")  
        print("2. Rechercher un produit par mot-clé")  
        print("3. Quitter")  
          
        choice = input("\nVotre choix (1-3): ")  
          
        if choice == '1':  
            # Sélectionner un produit aléatoire  
            sample_product_id = moroccan_df['Product Id'].sample(1).iloc[0]  
            sample_user_id = df['Uniq Id'].sample(1).iloc[0]  
            recommendations, product_info = get_recommendations(sample_product_id, sample_user_id)  
              
            print(f"\nProduit sélectionné: {product_info['Product Name']}")  
            print(f"Catégorie: {product_info['Product Category']}")  
            print(f"Description: {product_info['Product Description'][:200]}...")  
              
            # Affichage des recommandations  
            if 'svd' in recommendations:  
                print("\n📊 Recommandations par filtrage collaboratif (SVD):")  
                for i, (pid, score, name) in enumerate(recommendations['svd']):  
                    print(f"{i+1}. {name} (Score: {score:.4f})")  
              
            if 'tfidf' in recommendations:  
                print("\n📊 Recommandations par filtrage basé sur le contenu (TF-IDF):")  
                for i, (pid, score, name) in enumerate(recommendations['tfidf']):  
                    print(f"{i+1}. {name} (Similarité: {score:.4f})")  
              
            if 'bert' in recommendations:  
                print("\n📊 Recommandations par filtrage basé sur le contenu (BERT):")  
                for i, (pid, score, name) in enumerate(recommendations['bert']):  
                    print(f"{i+1}. {name} (Similarité: {score:.4f})")  
              
        elif choice == '2':  
            # Rechercher un produit par mot-clé  
            keyword = input("\nEntrez un mot-clé pour rechercher un produit: ")  
            matching_products = moroccan_df[moroccan  